In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd 
import shutil
import json
import re
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import torch.utils.data as data_utils
from torch.nn.modules import MSELoss, L1Loss

import sklearn.preprocessing
from sklearn.preprocessing import MultiLabelBinarizer
import glob
import csv
import cv2
import random
from PIL import Image
from itertools import product

In [ ]:
path1 = "./Movie_Poster_Metadata/groundtruth"
temp_path = "./Movie_Poster_Metadata/temp_groundtruth"
path2 = "./Movie_Poster_Metadata/updated_groundtruth"

In [ ]:
#append all json objects to a dataframe
dir_list = os.listdir(path2)
movies_df = pd.DataFrame()

for file_name in dir_list:
    df = pd.read_json(path2 + '/' + file_name, encoding = 'utf-8', orient='records')
    df = df[['imdbID', 'Director', 'Genre', 'imdbRating']]
    movies_df = pd.concat([movies_df,df], ignore_index= True)

In [ ]:
#remove not available genres
movies_df = movies_df[movies_df['Genre'] != 'N/A']
movies_df.shape

In [ ]:
movies_df = movies_df.drop_duplicates(subset= ['imdbID'], keep = 'last')
movies_df.set_index('imdbID', inplace = True)

In [ ]:
#needed to deal with space
genres = movies_df['Genre'].dropna().str.split(',').apply(lambda x: [genre.strip() for genre in x])

In [ ]:
all_genres = movies_df['Genre'].str.split(', ').explode()

# Count unique genres
unique_genres_count = all_genres.nunique()

In [ ]:
mlb = MultiLabelBinarizer()
# multihot_temp = mlb.fit_transform(movies_df['Genre'].dropna().str.split(','))
multihot_temp = mlb.fit_transform(genres)

In [ ]:
genres_df = pd.DataFrame({"multihot":[multihot_temp.astype(int)]}, index = movies_df.index)
movies_df = pd.concat([movies_df, genres_df], axis=1 )
print(mlb.classes_)

In [ ]:
#create a dictionary with multi-hot encoded vectors; index = imdbID
multihot_dict = {movies_df.index.tolist()[i] : multihot_temp[i] for i in range(0, len(multihot_temp))}

In [ ]:
classes = ['Action' ,'Adult', 'Adventure', 
           'Animation', 'Biography', 'Comedy', 'Crime',
            'Documentary', 'Drama', 'Family', 'Fantasy' ,
            'Game-Show', 'History', 'Horror',
            'Music', 'Musical', 'Mystery', 'News', 
            'Reality-TV', 'Romance', 'Sci-Fi',
            'Short', 'Sport', 'Talk-Show', 'Thriller', 'War', 'Western']

all_genres = [genre for genres in movies_df['Genre'].str.split(',').dropna() for genre in genres]

# Count the occurrences of each genre in the desired order
genre_counts = pd.Series(all_genres).value_counts().reindex(classes, fill_value=0)

# Create a bar chart for the genre counts before augmentation
plt.figure(figsize=(14, 7))
plt.bar(genre_counts.index, genre_counts.values, color='blue', width=0.4, align='center')

# Add numbers on top of the bars with smaller font size
for i, count in enumerate(genre_counts.values):
    plt.text(i, count + 2, str(count), ha='center', va='bottom', fontsize=8)

# Set chart title and labels with smaller font size
plt.title('Before Augmentation', fontsize=12)
plt.xlabel('Genre', fontsize=10)
plt.ylabel('Count', fontsize=10)

# Rotate x-axis labels for better visibility with smaller font size
plt.xticks(rotation=45, ha='right', fontsize=8)

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
#training controls
batch_size = 32
number_of_labels = 27
epochs = 5
training_size = 0.7
learning_rate = 0.001
dropout = [0.3, 0.3, 0.3, 0.3, 0.2, 0.2, 0.2, 0.2, 0.15]
# input image dimensions
img_rows, img_cols = 224, 224

In [ ]:
from glob import glob
x_train, y_train, x_test, y_test = [], [], [], []

# Specify the patterns for the two folders
pattern1 = './Movie_Poster_dataset/*/*.jpg'
pattern2 = './Movie_Poster_Dataset_Augmented/*.jpg'

# Use glob to get a list of file paths matching both patterns
flist = glob(pattern1) + glob(pattern2)
random.shuffle(flist)

genre_count = np.zeros(number_of_labels, dtype='float64')
length = int(len(flist)* training_size)

i = 0

for filename in flist:
    match = re.search(r'tt\d+', filename)
    if match:
        imdb_id = match.group()
    if imdb_id in multihot_dict:
        img = np.array(cv2.imread(filename)).transpose(2,0,1)
        genre_arr = np.array(multihot_dict[imdb_id])

        if i<length:
            x_train.append(img)
            y_train.append(genre_arr)
        else:
            x_test.append(img)
            y_test.append(genre_arr)
        # genre_count = genre_count.astype('float64')
        genre_count += genre_arr
        i += 1

classes = ['Action' ,'Adult', 'Adventure', 
           'Animation', 'Biography', 'Comedy', 'Crime',
            'Documentary', 'Drama', 'Family', 'Fantasy' ,
            'Game-Show', 'History', 'Horror',
            'Music', 'Musical', 'Mystery', 'News', 
            'Reality-TV', 'Romance', 'Sci-Fi',
            'Short', 'Sport', 'Talk-Show', 'Thriller', 'War', 'Western']



In [ ]:
img_sum, total  = 0,  0
for class_name, count in zip(classes, genre_count):
    print(f'{class_name}: {count}')
    img_sum += count
    total += count
print(f'total posters: {total}')

In [ ]:
# Set the figure size for a larger plot
plt.figure(figsize=(15, 7))

# Create a bar chart with adjusted width and alignment
plt.bar(classes, genre_count, color='blue', width=0.4, align='center')

# Add numbers on top of the bars with smaller font size
for i, count in enumerate(genre_count):
    plt.text(i, count + 2, str(count), ha='center', va='bottom', fontsize=8)

# Set chart title and labels with smaller font size
plt.title('After Augmentation', fontsize=12)
plt.xlabel('Class', fontsize=10)
plt.ylabel('Count', fontsize=10)

# Rotate x-axis labels for better visibility with smaller font size
plt.xticks(rotation=45, ha='right', fontsize=8)

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Function to plot an image
def plot_image(img, title):
    plt.imshow(img.transpose(1, 2, 0))
    plt.title(title)
    plt.axis('off')
    plt.show()

# Randomly select 5 samples from the test dataset
random_indices = np.random.choice(len(x_train), 5, replace=False)

for index in random_indices:
    img = x_train[index]
    imdb_id = re.search(r'tt\d+', flist[index]).group()
    genre_info = y_train[index]

    # Plot the image
    plot_image(img, f'IMDb ID: {imdb_id}')

    # Print corresponding genre information
    print(f'IMDb ID: {imdb_id}')
    print('Genre Information:')
    for class_name, genre_label in zip(classes, genre_info):
        if genre_label.any():
            print(f'- {class_name}')

    print('-' * 30)

In [ ]:
x_train[0].shape

In [ ]:
fixed_size = (32, 32)  # Adjust as needed
x_train_resized = [cv2.resize(img.transpose(1, 2, 0), fixed_size).transpose(2, 0, 1) for img in x_train]

x_test_resized = [cv2.resize(img.transpose(1, 2, 0), fixed_size).transpose(2, 0, 1) for img in x_test]

# Convert lists to NumPy arrays
x_train, y_train, x_test, y_test = map(np.array, [x_train_resized, y_train, x_test_resized, y_test])

In [ ]:
x_train = x_train.astype('float32')  # Convert to float32
x_test = x_test.astype('float32')  # Convert to float32


#scaling down the RGB data
x_train /= 255
x_test /= 255

#printing stats about the features
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

In [ ]:
import matplotlib.pyplot as plt

# Function to plot an image
def plot_image(img, title):
    plt.imshow(img.transpose(1, 2, 0))
    plt.title(title)
    plt.axis('off')
    plt.show()

# Randomly select 5 samples from the test dataset
random_indices = np.random.choice(len(x_train), 5, replace=False)

for index in random_indices:
    img = x_train_resized[index]
    imdb_id = re.search(r'tt\d+', flist[index]).group()
    genre_info = y_train[index]

    # Plot the image
    plot_image(img, f'IMDb ID: {imdb_id}')

    # Print corresponding genre information
    print(f'IMDb ID: {imdb_id}')
    print('Genre Information:')
    for class_name, genre_label in zip(classes, genre_info):
        if genre_label.any():
            print(f'- {class_name}')

    print('-' * 30)

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
# Assuming x_train_resized and x_test_resized are NumPy arrays
# Convert them to torch Tensors
x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Combine x and y tensors into a TensorDataset
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)

# Define batch size
batch_size = 64

# Create DataLoader for training and testing
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)


DenseNet-121

In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet121', pretrained=True)

for param in model.parameters():
    param.requires_grad = False

model.classifier = nn.Sequential(nn.Linear(1024, number_of_labels),
                                 nn.Sigmoid())

model.float()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"model loaded on {device}")
model.to(device) #hopefully runs model on cuda core.
print(model)

In [ ]:
for images, labels in train_loader:
    print(labels.shape)

Loss Function

In [ ]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate, weight_decay= 0.0)

Train the model 

In [ ]:
# Function to save the model
def saveModel():
    path = "./myFirstModel.pth"
    torch.save(model.state_dict(), path)

def testAccuracy(loader):
    model.eval()
    accuracy = 0
    total = 0
    
    classes = ['Action' ,'Adult', 'Adventure', 
           'Animation', 'Biography', 'Comedy', 'Crime',
            'Documentary', 'Drama', 'Family', 'Fantasy' ,
            'Game-Show', 'History', 'Horror',
            'Music', 'Musical', 'Mystery', 'News', 
            'Reality-TV', 'Romance', 'Sci-Fi',
            'Short', 'Sport', 'Talk-Show', 'Thriller', 'War', 'Western']

    class_correct = list(0 for i in range(number_of_labels))
    class_total = list(0 for i in range(number_of_labels))

    with torch.no_grad():
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        print(f"model loaded on {device}")
        init_accuracy, final_accuracy = 0,0 
        for images, labels in loader:
            sum = 0
            images, labels = images.to(device), labels.to(device)
            outputs = model(images) 

            for j, label in enumerate(labels):
                n = sum + label.sum().item()
                #get top-k outputs
                _, predicted = torch.topk(outputs[j], int(n))
                
                correct_predictions = 0
                for _, k in enumerate(predicted):
                    class_total[k] += 1
                    if(label[k].item() == 1):
                        correct_predictions += 1
                        class_correct[k] += 1
                    init_accuracy = correct_predictions/n
            final_accuracy += init_accuracy

            total += labels.size(0)
    print('Total:', total)

    #computer accuracy over all test images
    accuracy = final_accuracy*100 /total
    print('Accuracy:', accuracy)

    for i in range(number_of_labels):
        if class_total[i] != 0:
            print(i)
            print(class_correct[i], class_total[i])
            print('Accuracy of %5s: %2d %%'%(
                classes[i], 100 * class_correct[i]/ class_total[i]
                ))

    model.train()
    return(accuracy)

def train(num_epochs):
    loss_values = []
    accuracy_values = []
    best_accuracy = 0
    # Define your execution device
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("The model will be running on", device, "device")
    # Convert model parameters and buffers to CPU or Cuda
    model.to(device)

    for epoch in range(num_epochs):
        print("starting epoch", epoch)
        running_loss = 0
        running_acc = 0

        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(images)

            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i%1000 == 0:
                print('[%d, %5d] loss: %.3f'%
                      (epoch + 1, i+1, running_loss/1000))
                running_loss = 0

        accuracy = testAccuracy(train_loader)
        print('For epoch', epoch+1,'the test accuracy over the whole test set is %d %%' % (accuracy))

        # Append loss and accuracy values for plotting
        loss_values.append(running_loss / len(train_loader))
        accuracy_values.append(accuracy)

        if accuracy> best_accuracy:
            saveModel()
            best_accuracy = accuracy

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(loss_values, label='Training Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training Loss Over Epochs')
    plt.legend()

    # Plot accuracy values
    plt.subplot(1, 2, 2)
    plt.plot(accuracy_values, label='Test Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy (%)')
    plt.title('Test Accuracy Over Epochs')
    plt.legend()

    plt.tight_layout()
    plt.show()


In [ ]:
train(10)

test and plot some samples

In [ ]:
number_of_labels = 27
classes = ['Action' ,'Adult', 'Adventure', 
           'Animation', 'Biography', 'Comedy', 'Crime',
            'Documentary', 'Drama', 'Family', 'Fantasy' ,
            'Game-Show', 'History', 'Horror',
            'Music', 'Musical', 'Mystery', 'News', 
            'Reality-TV', 'Romance', 'Sci-Fi',
            'Short', 'Sport', 'Talk-Show', 'Thriller', 'War', 'Western']

# Assuming test_loader and model are defined

model.eval() 
total = 0
final_accuracy = 0

def save_sample(images, labels, predicted, genre_labels, save_path, index):
    for j in range(len(images)):
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.imshow(np.transpose(images[j].cpu().numpy(), (1, 2, 0)))
        plt.title('Actual genres: ' + ', '.join([genre_labels[l] for l in range(number_of_labels) if labels[j, l].item() == 1]))

        plt.subplot(1, 2, 2)
        plt.bar(range(number_of_labels), outputs[j].cpu().numpy())
        plt.title('Predicted genres: ' + ', '.join([genre_labels[l] for l in range(number_of_labels) if l in predicted[j]]))
        plt.show()

plot_counter = 0
with torch.no_grad():
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"model testing on {device}")
    

    for i, (images, labels) in enumerate(test_loader, 0):
        # if i not in indices:
        #     continue

        sum = 0
        images = Variable(images.to(device))
        labels = Variable(labels.to(device))
        outputs = model(images)
        
        for j, label in enumerate(labels):
            n = sum + label.sum().item()
            # get the top n values of outputs
            _, predicted = torch.topk(outputs[j], int(n))

            if plot_counter<5:   
            # Display images and genres
                plt.figure(figsize=(10, 5))
                plt.subplot(1, 2, 1)
                plt.imshow(np.transpose(images[j].cpu().numpy(), (1, 2, 0)))
                plt.title('Actual genres: ' + ', '.join([classes[l] for l in range(number_of_labels) if label[l].item() == 1]))

                plt.subplot(1, 2, 2)
                plt.bar(range(number_of_labels), outputs[j].cpu().numpy())
                plt.title('Predicted genres: ' + ', '.join([classes[l] for l in predicted]))

                plt.show()
                plot_counter+=1

        final_accuracy += some_accuracy_measure 
        total += labels.size(0)
        
    print("Total: ", total)
    if total != 0:
        # Compute the accuracy over all test images
        accuracy = final_accuracy * 100 / total
        print("Accuracy: ", accuracy)
